In [5]:
import pandas as pd
import os
import pickle
from datetime import datetime
from datetime import timezone

In [6]:
import json

config = json.load(open("../configure.json"))
start_date = datetime.fromisoformat(config['start_time']).astimezone(tz=timezone.utc).replace(tzinfo=None).timestamp()
end_date = datetime.fromisoformat(config['buffer_end_time']).astimezone(tz=timezone.utc).replace(
    tzinfo=None).timestamp()

In [7]:
charts = os.listdir('data/CHARTS')
length_of_time_windows = [5, 15, 30, 60, 240, 1440]
preferred_time = 1440
chart_pkls_path = config['chart_pkls']

In [8]:
form = "%d-%m-%Y %H:%M"


def company_chart(data_set):
    headers = ["date", "time", "open", "high", "low", "close", "volume"]
    stock_chart = pd.read_csv(data_set, names=headers)
    trends = stock_chart.apply(lambda row: 1 if row["close"] > row["open"] else -1, axis=1)
    timestamps = stock_chart.apply(
        lambda row: datetime.fromisoformat(row["date"].replace(".", "-") + " " + row["time"]).timestamp(), axis=1)
    time_frames = stock_chart.apply(
        lambda row: datetime.fromisoformat(row["date"].replace(".", "-") + " " + row["time"]).strftime(form), axis=1)
    df = pd.DataFrame({
        "timestamp": timestamps,
        "time_frame": time_frames,
        "trends": trends
    })
    df.drop(df[(df['timestamp'] < start_date) | (
            df['timestamp'] > end_date)].index,
            inplace=True)
    df.sort_values('timestamp', ascending=True, inplace=True)
    return df


for i in length_of_time_windows:
    amazon_stock_prices = company_chart('data/CHARTS/' + 'AMAZON' + str(i) + ".csv")
    apple_stock_prices = company_chart('data/CHARTS/' + 'APPLE' + str(i) + ".csv")
    with open(chart_pkls_path + 'amazon_stock_chart' + str(i) + '.pkl', 'wb') as f:
        pickle.dump(apple_stock_prices, f)
    with open(chart_pkls_path + 'apple_stock_chart' + str(i) + '.pkl', 'wb') as f:
        pickle.dump(apple_stock_prices, f)